# 1. BSGDA Sampling

In [1]:
import torch

from thgsp import Graph, loadmat
from thgsp.sampling.bsgda import bsgda, recon_bsgda
from thgsp.utils import mse, snr


def snr_and_mse(x, target):
    s, m = snr(x, target), mse(x, target)
    print(f"SNR: {s:.4f} | MSE: {m:.4e}")
    return s, m


dv = torch.device("cpu")
dt = torch.float

data = loadmat("bsgda.mat")
S = data["S"].ravel().astype(int) - 1

bw = data["bw"].item()
epsilon = data["epsilon"].item()
mu = data["mu"].item()
M = len(S)

f = torch.as_tensor(data["f"]).to(dt).to(dv)
y = torch.as_tensor(data["f_or"]).to(dt).to(dv)
L = Graph(data["L"]).to(dv, dt)
g = Graph(data["A"])
N = g.n_node

S2, _ = bsgda(L, M, mu, epsilon, 12, boost=False)
S1, _ = bsgda(L, M, mu, epsilon, 12)

print("MATLAB:       ", S)
print("Thgsp[boost]: ", S1)
print("Thgsp[nonbs]: ", S2)

f_hat = recon_bsgda(y[S], S, L, mu)
f_hat1 = recon_bsgda(y[S1], S1, L, mu)
f_hat2 = recon_bsgda(y[S1], S1, L, mu)
_, _ = snr_and_mse(f_hat, f)
_, _ = snr_and_mse(f_hat1, f)
_, _ = snr_and_mse(f_hat2, f)

/home/dengbw/anaconda3/envs/gsp/lib/python3.11/site-packages/scipy/io/matlab/_mio.py:235: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Considerscipy.io.matlab.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


MATLAB:        [ 0  1  2  3 13]
Thgsp[boost]:  [12, 8, 13, 15, 18]
Thgsp[nonbs]:  [12, 8, 13, 15, 18]
SNR: 2.7528 | MSE: 5.3097e-01
SNR: 2.1485 | MSE: 6.1026e-01
SNR: 2.1485 | MSE: 6.1026e-01


# 2. ESS Sampling

In [1]:
import torch

from thgsp import Graph, loadmat
from thgsp.sampling import ess, recon_ess
from thgsp.utils import mse, snr


def snr_and_mse(x, target):
    s, m = snr(x, target), mse(x, target)
    print(f"SNR: {s:.4f} | MSE: {m:.4e}")
    return s, m


dv = torch.device("cpu")
dt = torch.float

data = loadmat("ess-comm.mat")
S = data["S"].ravel().astype(int) - 1

bw = data["bw"][0, 0].astype(int)
order = data["K"].astype(int)[0, 0]
print(order)

graph = Graph(data["A"]).to(dv).to(dt)
f = torch.as_tensor(data["f"]).to(dt).to(dv)
y = torch.as_tensor(data["f_or"]).to(dt).to(dv)
L = graph.L("comb")
U = graph.U("comb")

M = len(S)
S1 = ess(L, M, k=order // 2)
print(S)
print(S1)

f_hat1 = recon_ess(y[S], S, U, bw)
f_hat2 = recon_ess(y[S1], S1, U, bw)

s, m = snr_and_mse(f_hat1, f.to(dv))
_, _ = snr_and_mse(f_hat2, f.to(dv))


4


/home/dengbw/projects/thgsp/thgsp/sampling/ess.py:53: UserWarning: Exited postprocessing with accuracies 
[  64.27013735 1318.71720258]
not reaching the requested tolerance 7.450580596923828e-06.
  sigma, psi = xsplin.lobpcg(reduced, X=guess, largest=False, maxiter=100)
/home/dengbw/projects/thgsp/thgsp/sampling/ess.py:53: UserWarning: Exited postprocessing with accuracies 
[113.23657167 872.00932225]
not reaching the requested tolerance 7.4356794357299805e-06.
  sigma, psi = xsplin.lobpcg(reduced, X=guess, largest=False, maxiter=100)
/home/dengbw/projects/thgsp/thgsp/sampling/ess.py:53: UserWarning: Exited postprocessing with accuracies 
[179.7252843  799.17889404]
not reaching the requested tolerance 7.420778274536133e-06.
  sigma, psi = xsplin.lobpcg(reduced, X=guess, largest=False, maxiter=100)
/home/dengbw/projects/thgsp/thgsp/sampling/ess.py:53: UserWarning: Exited postprocessing with accuracies 
[208.07884241 944.50608772]
not reaching the requested tolerance 7.405877113342285e-

[332  22 414 116 210 390 495 461 351 147 479 171 102 182  66 344 149 243
 323  84  18 409 465 338 426 491 205 172 419 397 155 322  21 175 448 490
 342  60 410 471 104 242 314 399  15 134 109 126 310 454  74 462 416 234
 494 196 377 296 415 183   7  69 315 160 440 387 433 485  13 349 396 469
  88 151 253 115 197 327 353 404 487  33 302 165 309  12 439 473 411 187
  44 341 133 468  30  37 380 121 489 405]
[351, 66, 383, 116, 495, 160, 323, 15, 404, 479, 409, 196, 171, 353, 302, 440, 149, 53, 205, 243, 114, 423, 465, 3, 349, 386, 491, 172, 332, 259, 155, 314, 12, 396, 461, 415, 322, 104, 494, 471, 242, 377, 69, 126, 487, 426, 13, 342, 296, 165, 134, 109, 462, 411, 7, 457, 298, 267, 485, 88, 416, 214, 147, 60, 18, 315, 87, 387, 234, 183, 388, 414, 463, 210, 448, 327, 151, 371, 344, 310, 22, 380, 37, 490, 410, 454, 84, 432, 21, 341, 133, 289, 33, 473, 30, 489, 372, 284, 318, 121]
SNR: 1.2057 | MSE: 7.5758e-01
SNR: 1.1673 | MSE: 7.6431e-01


/home/dengbw/projects/thgsp/thgsp/sampling/ess.py:53: UserWarning: Exited postprocessing with accuracies 
[1049.66177764 1001.67079357]
not reaching the requested tolerance 5.97536563873291e-06.
  sigma, psi = xsplin.lobpcg(reduced, X=guess, largest=False, maxiter=100)


# 3. RSBS Sampling

In [2]:
import torch
from torch_sparse import SparseTensor

from thgsp import loadmat
from thgsp.sampling.rsbs import recon_rsbs
from thgsp.utils import mse, snr


def snr_and_mse(x, target):
    s, m = snr(x, target), mse(x, target)
    print(f"SNR: {s:.4f} | MSE: {m:.4e}")
    return s, m


num_sig = 10  # repeat the signal for num_sig times


# Load Data captured from one trial of official code
# http://grsamplingbox.gforge.inria.fr.
# with the official code, the reconstruction SNR is about 30 dB, which is consistent
# with `rsbs` reconstruction in thgsp.


data = loadmat("rsbs.mat")
coh1 = data["weight"].ravel()  # distribution of nodes being sampled
S1 = data["ind_obs"].ravel().astype(int) - 1  # sampling node set
mu1 = data["mu"].item()  # the factor of regularization term
f = torch.as_tensor(data["x"])  # the original bandlimited signal
y = torch.as_tensor(data["ynoise_init"]).repeat(1, num_sig)  # the contaminated signal

L1 = SparseTensor.from_scipy(data["L"])  # the combinatorial laplacian used

f_hat = recon_rsbs(y, S=S1, L=L1, cum_coh=coh1, mu=mu1, reg_order=1)
s, m = snr_and_mse(f_hat.view(-1, num_sig), f)


SNR: 31.5076 | MSE: 7.0670e-07
